In [2]:
import numpy as np
import pandas as pd
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        """
        Initialize the Simplex Algorithm with data from a CSV file.

        Args:
            filePath (str): Path to the CSV file containing the simplex tableau.
        """
        # Read the CSV file without headers
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        
        # Extract vectors and matrix based on the input format
        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial feasible point (shape: (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Cost vector (shape: (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint bounds (shape: (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64)  # Constraint matrix (shape: (m, n))
        
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints
        
        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")
        
        self.b_original = self.b.copy()  # Save the original b for perturbation
        self.epsilon = 1e-5  # Initial perturbation factor to handle degeneracy

        # Check and resolve degeneracy during initialization
        print("Checking for degeneracy...")
        if self.is_degenerate():
            self.apply_perturbation()
            print("Degeneracy resolved with perturbation.")
        else:
            print("No degeneracy detected.")

    def isFeasible(self, z=None):
        """
        Check if the current solution z is feasible.

        Args:
            z (np.ndarray, optional): Solution vector to check. Defaults to self.z.

        Returns:
            bool: True if feasible, False otherwise.
        """
        if z is None:
            z = self.z
        return np.all(np.matmul(self.A, z) <= self.b + 1e-8)  # Added tolerance for numerical stability

    def classify_constraints(self, z):
        """
        Classify constraints as active or inactive based on the current z.

        Args:
            z (np.ndarray): Current solution vector.
        """
        constraint_values = np.matmul(self.A, z)
        tolerance = 1e-6  # Tolerance for determining active constraints
        active_indices = np.where(np.abs(constraint_values - self.b) <= tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - tolerance)[0]
        
        self.A1 = self.A[active_indices]
        self.b1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.b2 = self.b[inactive_indices]
        self.active_indices = active_indices
        self.inactive_indices = inactive_indices

    def is_degenerate(self):
        """
        Check if the current problem is degenerate.

        Returns:
            bool: True if degenerate, False otherwise.
        """
        rank = np.linalg.matrix_rank(self.A)
        is_degenerate = rank < min(self.A.shape)
        print(f"Checking degeneracy: Rank={rank}, Full rank required={min(self.A.shape)}")
        return is_degenerate

    def apply_perturbation(self):
        """
        Apply the perturbation method to make the problem non-degenerate by adjusting the b vector.
        """
        perturbation = np.array([self.epsilon ** i for i in range(1, len(self.b) + 1)])
        self.b += perturbation
        print(f"Perturbation applied to b: {perturbation}")
        print(f"New b after perturbation: {self.b}")

        if self.is_degenerate():
            self.epsilon *= 10
            self.apply_perturbation()

    def _isVertex(self, z):
        """
        Check if the current solution z is a vertex.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if z is a vertex, False otherwise.
        """
        self.classify_constraints(z)
        
        if self.A1.shape[0] < self.n:
            print("Not enough active constraints to form a vertex.")
            return False
        if self.A1.shape[0] > self.n:
            # More active constraints than variables may indicate degeneracy
            print("More active constraints than variables; potential degeneracy.")
            return False
        
        # Check if the active constraints matrix has full rank
        rank = np.linalg.matrix_rank(self.A1)
        print(f"Rank of active constraints: {rank} (required: {self.n})")
        if rank < self.n:
            # Active constraints do not form a full rank matrix
            print("Active constraints do not form a full rank matrix.")
            return False
        
        return True

    def move_to_vertex(self, z):
        """
        Move the current solution z to a valid vertex by considering directions in the nullspace.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            np.ndarray: Updated solution vector after moving towards a vertex.
        """
        while not self._isVertex(z):
            self.classify_constraints(z)
            print(f"Active Constraints: {self.A1.shape}, Inactive Constraints: {self.A2.shape}")

            if self.A1.size == 0:
                print("No active constraints to form a vertex. Selecting all variables to move.")
                nullspace = np.eye(self.n)
            else:
                # Compute the nullspace of the active constraints matrix
                nullspace = np.array(Matrix(self.A1).nullspace())
                nullspace = np.asarray(nullspace, dtype=np.float64)

            if nullspace.size == 0:
                print("Already at a vertex.")
                break  # Already at a vertex

            # Iterate through possible directions in the nullspace
            for ns_vec in nullspace:
                for direction in [ns_vec, -ns_vec]:
                    direction = direction.flatten()
                    if np.linalg.norm(direction) == 0:
                        continue
                    direction = direction / np.linalg.norm(direction)  # Normalize direction

                    print(f"Trying direction: {direction}")

                    # Compute potential step sizes (alpha)
                    temp = np.dot(self.A2, direction)
                    valid_indices = temp > 1e-8  # Only consider constraints that allow movement in this direction

                    if not np.any(valid_indices):
                        continue  # No valid movement in this direction

                    alpha = (self.b2 - np.dot(self.A2, z)) / temp
                    alpha = alpha[valid_indices]
                    alpha = alpha[alpha > 1e-8]

                    if alpha.size == 0:
                        continue  # No valid positive alpha found

                    alpha_min = np.min(alpha)
                    print(f"Found alpha={alpha_min} for direction={direction}")

                    # Update z
                    z_new = z + alpha_min * direction
                    print(f"Moving to new z: {z_new}")
                    return z_new

            # If no valid direction found, declare no movement possible
            raise ValueError("No valid direction found to move towards a vertex.")

        return z

    def solve(self):
        """
        Solve the linear programming problem using the simplex algorithm.
        """
        # Step 1: Check for feasibility of the Initial Point
        if not self.isFeasible():
            print(f"Initial point {self.z} is not feasible...!!!")
            return

        solved = False
        iteration = 0
        max_iterations = 1000  # Prevent infinite loops

        current_z = self.z.copy()

        while not solved and iteration < max_iterations:
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")

            # Step 2: Move towards the first vertex
            try:
                current_z = self.move_to_vertex(current_z)
            except ValueError as e:
                print(f"Error during moving to vertex: {e}")
                return

            # Test for degeneracy of the vertex
            if self.is_degenerate():
                print("Degenerate vertex detected. Applying perturbation...")
                self.apply_perturbation()
                continue  # Re-evaluate feasibility and vertex status after perturbation

            # Print the value of the Cost function at this vertex
            current_cost = np.dot(current_z, self.c)
            print(f"Vertex {current_z} --> Cost: {current_cost}")

            # Check for optimality
            if self.test_optimality(current_z):
                print(f"Optimal solution found: Vertex {current_z} --> Cost: {current_cost}")
                solved = True

    def test_optimality(self, z):
        """
        Test if the current vertex is optimal.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if optimal, False otherwise.
        """
        self.classify_constraints(z)
        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            raise ValueError("Active constraint matrix A1 is singular.")

        beta = np.dot(A1_inv.T, self.c)
        print(f"Beta values: {beta}")
        return np.all(beta >= -1e-6)  # Using a small tolerance for numerical stability

    def test_optimality(self, z):
        """
        Test if the current vertex is optimal.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if optimal, False otherwise.
        """
        self.classify_constraints(z)
        
        # Ensure A1 is square before inversion
        if self.A1.shape[0] != self.A1.shape[1]:
            raise ValueError("Active constraint matrix A1 is not square. Cannot proceed with inversion.")
        
        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            raise ValueError("Active constraint matrix A1 is singular.")

        beta = np.dot(A1_inv.T, self.c)
        print(f"Beta values: {beta}")
        return np.all(beta >= -1e-6)  # Using a small tolerance for numerical stability

# Example Usage
if __name__ == "__main__":
    # Replace the file path with the path to your CSV input file
    file_path = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv'
    simplex = SimplexAlgo(file_path)
    simplex.solve()


Checking for degeneracy...
Checking degeneracy: Rank=2, Full rank required=2
No degeneracy detected.

--- Iteration 1 ---
Not enough active constraints to form a vertex.
Active Constraints: (1, 2), Inactive Constraints: (3, 2)
Trying direction: [0. 1.]
Trying direction: [-0. -1.]
Found alpha=2.0 for direction=[-0. -1.]
Moving to new z: [1. 0.]
Checking degeneracy: Rank=2, Full rank required=2
Vertex [1. 0.] --> Cost: -3.0


C:\Users\gupta\AppData\Local\Temp\ipykernel_41528\2880640558.py:170: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.b2 - np.dot(self.A2, z)) / temp


ValueError: Active constraint matrix A1 is singular.

In [3]:
import numpy as np
import pandas as pd

def simplex_algorithm(file_path):
    # Load the CSV file
    data = pd.read_csv(file_path, header=None).values
    
    # Parse the input
    initial_feasible_point = data[0, :-1]
    cost_vector = data[1, :-1]
    constraint_vector = data[2:, -1]
    A = data[2:, :-1]
    
    m, n = A.shape  # m constraints, n variables
    
    # Add slack variables to convert inequalities into equalities
    slack = np.eye(m)
    A = np.hstack([A, slack])  # Augment the constraint matrix with slack variables
    cost_vector = np.concatenate([cost_vector, np.zeros(m)])  # Add zero costs for slack variables
    
    # Initialize the basis with slack variable indices
    basis = list(range(n, n + m))
    non_basis = list(range(n))
    
    # Compute the initial basic feasible solution
    B = A[:, basis]
    N = A[:, non_basis]
    x_b = np.linalg.solve(B, constraint_vector)
    
    # Check feasibility of the initial solution
    if np.any(x_b < 0):
        raise ValueError("Initial solution is not feasible. Ensure the input data satisfies the constraints.")
    
    vertices_visited = []
    objective_values = []
    
    while True:
        # Update basis and non-basis matrices
        c_b = cost_vector[basis]
        c_n = cost_vector[non_basis]
        
        # Compute reduced costs
        p = np.linalg.solve(B.T, c_b)
        reduced_costs = c_n - N.T @ p
        
        # Check for optimality
        if np.all(reduced_costs <= 0):
            break
        
        # Determine entering variable (most positive reduced cost)
        entering = np.argmax(reduced_costs)
        d = np.linalg.solve(B, A[:, non_basis[entering]])
        
        # Check for unboundedness
        if np.all(d <= 0):
            raise ValueError("The problem is unbounded.")
        
        # Compute ratios for the leaving variable
        ratios = np.divide(x_b, d, out=np.full_like(x_b, np.inf), where=d > 0)
        leaving = np.argmin(ratios)
        
        # Update basis and non-basis indices
        leaving_var = basis[leaving]
        entering_var = non_basis[entering]
        basis[leaving] = entering_var
        non_basis[entering] = leaving_var
        
        # Update B, N, and x_b
        B = A[:, basis]
        N = A[:, non_basis]
        x_b = np.linalg.solve(B, constraint_vector)
        
        # Store the vertex and objective value
        vertex = np.zeros(n + m)
        vertex[basis] = x_b
        vertices_visited.append(vertex[:n])  # Only store the original variables
        objective_values.append(c_b @ x_b)
    
    # Output results
    for i, vertex in enumerate(vertices_visited):
        print(f"Vertex {i + 1}: {vertex}, Objective value: {objective_values[i]}")

# Example usage
# Replace 'your_file.csv' with the path to your CSV file
file_path = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv'
simplex_algorithm(file_path)


ValueError: Initial solution is not feasible. Ensure the input data satisfies the constraints.

In [11]:
import numpy as np
import pandas as pd
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        self.z = arr.iloc[0, :-1].values.astype(np.float64).flatten()
        self.c = arr.iloc[1, :-1].values.astype(np.float64).flatten()
        self.b = arr.iloc[2:, -1].values.astype(np.float64).flatten()
        self.A = arr.iloc[2:, :-1].values.astype(np.float64)

        self.n = self.A.shape[1]
        self.m = self.A.shape[0]
        self.b_original = self.b.copy()
        self.epsilon = 1e-5

    def isFeasible(self, z=None):
        if z is None:
            z = self.z
        z = np.array(z, dtype=float).flatten()  # Ensure z is 1D
        return np.all(np.matmul(self.A, z) <= self.b + 1e-8)

    def classifyConstraints(self, z):
        z = np.array(z, dtype=float).flatten()  # Ensure z is 1D
        constraints = np.matmul(self.A, z)
        active_indices = np.where(np.isclose(constraints, self.b))[0]
        inactive_indices = np.where(constraints < self.b)[0]
        self.A1 = self.A[active_indices]
        self.b1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.b2 = self.b[inactive_indices]
        return active_indices, inactive_indices

    def moveToVertex(self, z):
        z = np.array(z, dtype=float).flatten()  # Ensure z is 1D
        while True:
            self.classifyConstraints(z)
            nullspace = np.array(Matrix(self.A1).nullspace())
            if nullspace.size == 0:
                print("Already at a vertex or no nullspace directions available.")
                return z

            # Choose the first direction in the nullspace
            direction = nullspace[0].astype(float).flatten()
            direction = direction / np.linalg.norm(direction)

            temp = np.dot(self.A2, direction)
            valid_indices = temp > 0

            if not np.any(valid_indices):
                print("No valid directions to move. Staying at the current vertex.")
                return z

            alpha = (self.b2 - np.dot(self.A2, z)) / temp
            alpha = alpha[valid_indices]

            if alpha.size == 0:
                print("No positive alpha values found. Staying at the current vertex.")
                return z

            alpha_min = alpha.min()
            z += alpha_min * direction
            print(f"Moved to new point: {z}")
            return z

    def optimize(self, z):
        z = np.array(z, dtype=float).flatten()  # Ensure z is 1D
        self.classifyConstraints(z)

        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            print("Active constraints matrix is singular. Cannot proceed.")
            return z

        beta = np.dot(A1_inv.T, self.c)
        entering_var = np.argmin(beta)  # Find the variable to enter the basis

        if beta[entering_var] >= 0:
            print("Optimal solution found. No negative beta values.")
            return z

        direction = A1_inv[:, entering_var].flatten()
        temp = np.dot(self.A2, direction)
        valid_indices = temp > 0

        if not np.any(valid_indices):
            print("No valid directions to optimize. Staying at the current point.")
            return z

        # Calculate step sizes and ensure it's not empty
        alpha = (self.b2 - np.dot(self.A2, z)) / temp
        alpha = alpha[valid_indices]

        if alpha.size == 0:
            print("No positive alpha values found for optimization.")
            return z

        alpha_min = alpha.min()
        z += alpha_min * direction
        print(f"Optimized to new point: {z}")
        return z


    def testDegeneracy(self, z):
        z = np.array(z, dtype=float).flatten()  # Ensure z is 1D
        self.classifyConstraints(z)
        return np.linalg.matrix_rank(self.A1) < self.A1.shape[0]

    def removeDegeneracy(self):
        perturb = self.epsilon ** np.arange(len(self.b_original))
        self.b = self.b_original + perturb
        self.epsilon *= 2

    def testOptimum(self, z):
        z = np.array(z, dtype=float).flatten()  # Ensure z is 1D
        self.classifyConstraints(z)
        A1_inv = np.linalg.pinv(self.A1)
        beta = np.dot(A1_inv.T, self.c)
        return np.all(beta >= 0)


    def solve(self):
        if not self.isFeasible():
            print("Initial point is not feasible.")
            return
        z = self.z.copy()
        while True:
            z = self.moveToVertex(z)
            if self.testDegeneracy(z):
                self.removeDegeneracy()
                continue
            if self.testOptimum(z):
                print(f"Optimal solution: {z} with cost {np.dot(self.c, z)}")
                return
            z = self.optimize(z)

if __name__ == "__main__":
    # Replace 'example.csv' with the path to your CSV input file
    # file_path = "example.csv"
    file_path = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\2.Dataset.csv'
    
    
    # Initialize the Simplex Algorithm with the given file
    simplex_solver = SimplexAlgo(file_path)
    
    # Solve the linear optimization problem
    simplex_solver.solve()


No valid directions to move. Staying at the current vertex.


ValueError: zero-size array to reduction operation maximum which has no identity

In [12]:
import numpy as np
import pandas as pd
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        """
        Initialize the Simplex Algorithm with data from a CSV file.

        Args:
            filePath (str): Path to the CSV file containing the simplex tableau.
        """
        # Read the CSV file without headers
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        
        # Extract vectors and matrix based on the input format
        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial feasible point (shape: (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Cost vector (shape: (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint bounds (shape: (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64)  # Constraint matrix (shape: (m, n))
        
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints
        
        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")
        
        self.b_original = self.b.copy()  # Save the original b for perturbation
        self.epsilon = 1e-8  # Initial perturbation factor to handle degeneracy

    def isFeasible(self, z=None):
        """
        Check if the current solution z is feasible.

        Args:
            z (np.ndarray, optional): Solution vector to check. Defaults to self.z.

        Returns:
            bool: True if feasible, False otherwise.
        """
        if z is None:
            z = self.z
        return np.all(np.matmul(self.A, z) <= self.b + 1e-8)  # Added tolerance for numerical stability

    def __classify_constraints(self, z):
        """
        Classify constraints as active or inactive based on the current z.

        Args:
            z (np.ndarray): Current solution vector.
        """
        constraint_values = np.matmul(self.A, z)
        tolerance = 1e-8  # Tolerance for determining active constraints
        active_indices = np.where(np.abs(constraint_values - self.b) <= tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - tolerance)[0]
        
        self.A1 = self.A[active_indices]
        self.b1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.b2 = self.b[inactive_indices]
        self.active_indices = active_indices
        self.inactive_indices = inactive_indices
        
        print(f"Classified {len(active_indices)} active constraints and {len(inactive_indices)} inactive constraints.")

    def __isVertex(self, z):
        """
        Check if the current solution z is a vertex.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if z is a vertex, False otherwise.
        """
        self.__classify_constraints(z)
        
        if self.A1.shape[0] != self.n:
            print("Number of active constraints does not equal the number of variables.")
            return False

        rank = np.linalg.matrix_rank(self.A1)
        print(f"Rank of active constraints: {rank} (required: {self.n})")
        if rank < self.n:
            print("Active constraints do not form a full rank matrix.")
            return False
        
        return True

    def __testDegeneracy(self, z):
        """
        Test if the current vertex is degenerate.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if degenerate, False otherwise.
        """
        self.__classify_constraints(z)
        rank = np.linalg.matrix_rank(self.A1)
        is_degenerate = rank < self.A1.shape[0]
        print(f"Testing degeneracy: Rank={rank}, Number of active constraints={self.A1.shape[0]} -> {'Degenerate' if is_degenerate else 'Non-degenerate'}")
        return is_degenerate

    def __removeDegeneracy(self):
        """
        Perturb b slightly to handle degeneracy by adding a small epsilon to active constraints.
        """
        perturbation = self.epsilon * np.ones(len(self.active_indices))
        self.b[self.active_indices] = self.b_original[self.active_indices] + perturbation
        print(f"Perturbing b for active constraints with epsilon={self.epsilon}: {self.b[self.active_indices]}")
        self.epsilon *= 0.1  # Decrease epsilon for future perturbations if needed

    def __moveToVertex(self, z):
        """
        Move the current solution z to a valid vertex by considering directions in the nullspace.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            np.ndarray: Updated solution vector after moving towards a vertex.
        """
        while not self.__isVertex(z):
            self.__classify_constraints(z)
            print(f"Active Constraints: {self.A1.shape}, Inactive Constraints: {self.A2.shape}")

            if self.A1.shape[0] == 0:
                # Nullspace is the entire space
                nullspace = np.eye(self.n)
            else:
                # Compute nullspace of A1
                nullspace_sym = Matrix(self.A1).nullspace()
                if len(nullspace_sym) == 0:
                    print("Already at a vertex.")
                    break  # Already at a vertex
                nullspace = np.hstack([np.array(vec).astype(np.float64) for vec in nullspace_sym])

            # Iterate over possible directions in the nullspace
            for i in range(nullspace.shape[1]):
                direction = nullspace[:, i]
                direction = direction.flatten()
                if np.linalg.norm(direction) == 0:
                    continue
                direction = direction / np.linalg.norm(direction)  # Normalize direction

                print(f"Trying direction: {direction}")

                temp = np.dot(self.A2, direction)
                # Only consider directions where temp < 0 (moving towards constraints)
                valid_indices = temp < -1e-8

                if not np.any(valid_indices):
                    continue  # No valid movement in this direction

                alpha = (self.b2 - np.dot(self.A2, z)) / temp
                alpha = alpha[valid_indices]
                alpha = alpha[alpha > 1e-8]

                if alpha.size == 0:
                    continue  # No valid positive alpha found

                alpha_min = np.min(alpha)
                print(f"Found alpha={alpha_min} for direction={direction}")

                # Update z
                z_new = z + alpha_min * direction
                if self.isFeasible(z_new):
                    print(f"Moving to new z: {z_new}")
                    return z_new
                else:
                    print("New z is not feasible after moving.")

            # If no valid direction found, declare no movement possible
            raise ValueError("No valid direction found to move towards a vertex.")

        return z

    def __testOptimum(self, z):
        """
        Test if the current vertex is optimal.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if optimal, False otherwise.
        """
        self.__classify_constraints(z)

        if self.A1.shape[0] != self.n:
            print("Cannot test optimality: Number of active constraints does not equal number of variables.")
            return False

        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            print("Active constraint matrix A1 is singular. Cannot test optimality.")
            return False

        beta = np.dot(A1_inv.T, self.c)
        print(f"Beta values: {beta}")

        is_optimal = np.all(beta >= -1e-8)  # Using a small tolerance for numerical stability
        print(f"Testing optimality: {'Optimal' if is_optimal else 'Not optimal'}")
        return is_optimal

    def __optimize(self, z):
        """
        Optimize from the current vertex to find the next optimal vertex.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            tuple: (Updated solution vector, True if optimization should continue, False otherwise)
        """
        self.__classify_constraints(z)

        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            raise ValueError("Active constraint matrix A1 is singular.")

        beta = np.dot(A1_inv.T, self.c)
        print(f"Beta values for optimization: {beta}")

        # Find indices where beta is negative
        negative_beta_indices = np.where(beta < -1e-8)[0]

        if negative_beta_indices.size == 0:
            # All beta >= 0; current vertex is optimal
            return z, False

        for idx in negative_beta_indices:
            # Determine the direction to move
            u = -A1_inv[:, idx]
            u = u.flatten()
            if np.linalg.norm(u) == 0:
                continue
            u = u / np.linalg.norm(u)  # Normalize direction
            print(f"Attempting to move in direction: {u}")

            temp = np.dot(self.A2, u)
            # Only consider directions where temp < 0 (moving towards constraints)
            valid_indices = temp < -1e-8

            if not np.any(valid_indices):
                print("No valid movement in this direction. Continuing to next.")
                continue

            alpha = (self.b2 - np.dot(self.A2, z)) / temp
            alpha = alpha[valid_indices]
            alpha = alpha[alpha > 1e-8]

            if alpha.size == 0:
                print("No positive alpha found in this direction. Continuing to next.")
                continue

            alpha_min = np.min(alpha)
            z_new = z + alpha_min * u
            if self.isFeasible(z_new):
                print(f"Moving to new z: {z_new}")
                return z_new, True
            else:
                print("New z is not feasible after moving. Continuing to next direction.")

        # No valid direction found
        print("No feasible movement found for any entering variable. Declaring current vertex as optimal.")
        return z, False

    def solve(self):
        """
        Solve the linear programming problem using the simplex algorithm.
        """
        # Step 1: Check for feasibility of the Initial Point
        if not self.isFeasible():
            print(f"Initial point {self.z} is not feasible...!!!")
            return

        solved = False
        iteration = 0
        max_iterations = 1000  # Prevent infinite loops

        current_z = self.z.copy()

        while not solved and iteration < max_iterations:
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")

            # Step 2: Move towards the first vertex
            try:
                current_z = self.__moveToVertex(current_z)
            except ValueError as e:
                print(f"Error during moving to vertex: {e}")
                return

            # Test for degeneracy of the vertex
            if self.__testDegeneracy(current_z):
                print("Degenerate vertex detected. Applying perturbation...")
                self.__removeDegeneracy()
                # Restart the process with perturbed b
                current_z = self.z.copy()
                continue  # Re-evaluate feasibility and vertex status after perturbation

            # Print the value of the Cost function at this vertex
            current_cost = np.dot(current_z, self.c)
            print(f"Vertex {current_z} --> Cost: {current_cost}")

            # Step 3: Optimize to find the next optimal vertex
            while True:
                is_optimal = self.__testOptimum(current_z)
                if is_optimal:
                    print(f"Optimal solution found: Vertex {current_z} --> Cost: {current_cost}")
                    solved = True
                    break

                # Optimize to find the next vertex
                try:
                    new_z, continue_opt = self.__optimize(current_z)
                except ValueError as e:
                    print(f"Error during optimization: {e}")
                    return

                if not continue_opt:
                    current_z = new_z
                    current_cost = np.dot(current_z, self.c)
                    print(f"Optimal solution found: Vertex {current_z} --> Cost: {current_cost}")
                    solved = True
                    break

                # Update current_z for the next iteration
                current_z = new_z
                current_cost = np.dot(current_z, self.c)
                print(f"Vertex {current_z} --> Cost: {current_cost}")

            if iteration == max_iterations:
                print("Maximum iterations reached. Possible cycling or no feasible solution.")
                return

        if not solved:
            print("Failed to find an optimal solution.")
            return

        # Assign the found optimal solution
        self.z = current_z

        # Display the original solution if possible
        X, y = [], []
        for i in range(len(self.A)):
            if np.abs(np.dot(self.A[i], self.z) - self.b_original[i]) <= 1e-6:
                X.append(self.A[i])
                y.append(self.b_original[i])

        if len(X) >= self.n:
            try:
                X = np.array(X[:self.n])
                y = np.array(y[:self.n])
                z_original = np.linalg.solve(X, y)
                print(f"Original Solution: Vertex {z_original} --> Cost: {np.dot(z_original, self.c)}")
            except np.linalg.LinAlgError:
                print("Cannot find original solution due to singular matrix.")
        else:
            print("Cannot reconstruct the original solution. Insufficient active constraints.")

        print(f"Final Optimal Solution: Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}")

# Example Usage
if __name__ == "__main__":
    # Replace the file path with the path to your CSV input file
    
    file_path = 'your_file_path.csv'  # Replace with your CSV file path
    file_path = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\2.Dataset.csv'

    simplex = SimplexAlgo(file_path)
    simplex.solve()



--- Iteration 1 ---
Classified 0 active constraints and 3 inactive constraints.
Number of active constraints does not equal the number of variables.
Classified 0 active constraints and 3 inactive constraints.
Active Constraints: (0, 2), Inactive Constraints: (3, 2)
Trying direction: [1. 0.]
Trying direction: [0. 1.]
Error during moving to vertex: No valid direction found to move towards a vertex.


C:\Users\gupta\AppData\Local\Temp\ipykernel_41528\983217387.py:157: RuntimeWarning: divide by zero encountered in divide
  alpha = (self.b2 - np.dot(self.A2, z)) / temp


In [13]:
import numpy as np
from sympy import Matrix, zoo

def readCSV(filename):
    with open(filename) as input:
        line = input.readline()
        z = list(map(float, line.split(',')))
        z = np.asarray(z[:-1], dtype=float)
        print(f"Initial point z: {z}")

        line = input.readline()
        c = list(map(float, line.split(',')))
        c = np.asarray(c[:-1], dtype=float)
        print(f"Cost vector c: {c}")

        A, b = [], []
        for line in input.readlines():
            l = list(map(float, line.split(',')))
            A.append(l[:-1])
            b.append(l[-1])
        A = np.asarray(A, dtype=float)
        b = np.asarray(b, dtype=float)

        print(f"Constraint matrix A:\n{A}")
        print(f"Constraint vector b: {b}")
    return A, b, c, z

def testFeasibility(A, b, z):
    feasibility = all(np.matmul(A, z) <= b)
    print(f"Feasibility of point {z}: {feasibility}")
    return feasibility

def splitRows(A, b, z):
    A1, A2, b1, b2 = [], [], [], []
    Az = np.dot(A, z)

    for i in range(len(Az)):
        if np.dot(A[i], z) == b[i]:
            A1.append(A[i])
            b1.append(b[i])
        else:
            A2.append(A[i])
            b2.append(b[i])

    A1, A2 = np.array(A1), np.array(A2)
    b1, b2 = np.array(b1), np.array(b2)
    print(f"Tight rows A1:\n{A1}")
    print(f"Tight rows b1: {b1}")
    print(f"Untight rows A2:\n{A2}")
    print(f"Untight rows b2: {b2}")
    return A1, b1, A2, b2

def testVertex(A, b, z):
    A1, _, _, _ = splitRows(A, b, z)
    is_vertex = np.linalg.matrix_rank(A) <= A1.shape[0]
    print(f"Point {z} is vertex: {is_vertex}")
    return is_vertex

def move(A, b, z):
    _, n = A.shape[0], A.shape[1]
    A1, _, A2, b2 = splitRows(A, b, z)
    A1 = A1.reshape((A1.shape[0], n))

    nullspace = np.array(Matrix(A1).nullspace())
    scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
    vector = np.array(np.sum(nullspace * scale, axis=0)).astype(float)
    u = vector / np.linalg.norm(vector)

    temp = np.dot(A2, u)
    for i in range(len(temp)):
        if temp[i] == 0:
            temp[i] = 1e-32
    alpha = (b2 - np.dot(A2, z)) / temp.T
    alpha = alpha[0]
    alpha = np.array([a for a in alpha if not ((a > 10**6) or (-1 * a) > 10**6)])
    alpha = alpha[np.argmin(np.abs(alpha.copy()))]

    u = u.T
    z = z + alpha * u
    z = z[0]
    print(f"Updated point z after move: {z}")
    return z

def testDegeneracy(A, b, z):
    A1, _, _, _ = splitRows(A, b, z)
    is_degenerate = np.linalg.matrix_rank(A) < A1.shape[0]
    print(f"Vertex {z} is degenerate: {is_degenerate}")
    return is_degenerate

def removeDegeneracy(b, epsilon):
    add = epsilon
    for i in range(len(b)):
        b[i] += add
        add *= epsilon
    print(f"Vector b after removing degeneracy: {b}")
    return b

def testOptimum(A, b, z, c):
    A1, _, _, _ = splitRows(A, b, z)
    beta = np.dot(np.linalg.inv(A1.T), c.T)
    is_optimum = all([x >= 0 for x in beta])
    print(f"Vertex {z} is optimum: {is_optimum}")
    return is_optimum

def optimize(A, b, z, c):
    A1, _, A2, b2 = splitRows(A, b, z)
    u = None
    A1_inverse = np.linalg.inv(A1)
    A1_inverse = np.transpose(A1_inverse)
    for x in A1_inverse:
        if np.dot((-1) * x, c.T) > 0:
            u = x.T
            break

    temp = np.dot(A2, u)
    for i in range(len(temp)):
        if temp[i] == 0:
            temp[i] = 1e-16
    alpha = (np.dot(A2, z) - b2) / temp
    alpha = [x for x in alpha if x != zoo]
    if all([a < 0 for a in alpha]):
        return z, False
    alpha = min([x for x in alpha if x >= 0])
    z = z - alpha * u
    print(f"Next vertex z after optimization: {z}")
    return z, True

def main():
    A, b, c, z = readCSV(r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv')
    b_org = b.copy()
    if not testFeasibility(A, b, z):
        print(f'Initial point {z} is not feasible.')
        return

    solved = False
    epsilon = pow(2, -1)

    while not solved:
        while not testVertex(A, b, z):
            z = move(A, b, z)

        if testDegeneracy(A, b, z):
            b = removeDegeneracy(b_org.copy(), epsilon)
            epsilon *= 2
            continue

        print(f'Vertex {z} --> Cost: {np.dot(z, c)}')

        solution = True
        degenerate = False
        while solution and not testOptimum(A, b, z, c):
            z, solution = optimize(A, b, z, c)
            if not solution:
                print('No solution.')
            elif testDegeneracy(A, b, z):
                b = removeDegeneracy(b_org.copy(), epsilon)
                degenerate = True
                epsilon *= 2
                break
            else:
                print(f'Vertex {z} --> Cost: {np.dot(z, c)}')

        if not degenerate:
            solved = True

    if solution:
        X, y = [], []
        for i in range(len(A)):
            if np.dot(A[i], z) == b[i]:
                X.append(A[i])
                y.append(b_org[i])
        z = np.dot(np.linalg.inv(X), y)
        print(f'Original Solution: Vertex {z} --> Cost: {np.dot(z, c)}')

main()


Initial point z: [1. 2.]
Cost vector c: [-3. -4.]
Constraint matrix A:
[[-1. -1.]
 [-1.  0.]
 [ 0. -1.]
 [ 1.  0.]]
Constraint vector b: [-1.  0.  0.  1.]
Feasibility of point [1. 2.]: True
Tight rows A1:
[[1. 0.]]
Tight rows b1: [1.]
Untight rows A2:
[[-1. -1.]
 [-1.  0.]
 [ 0. -1.]]
Untight rows b2: [-1.  0.  0.]
Point [1. 2.] is vertex: False
Tight rows A1:
[[1. 0.]]
Tight rows b1: [1.]
Untight rows A2:
[[-1. -1.]
 [-1.  0.]
 [ 0. -1.]]
Untight rows b2: [-1.  0.  0.]
Updated point z after move: [1. 0.]
Tight rows A1:
[[-1. -1.]
 [ 0. -1.]
 [ 1.  0.]]
Tight rows b1: [-1.  0.  1.]
Untight rows A2:
[[-1.  0.]]
Untight rows b2: [0.]
Point [1. 0.] is vertex: True
Tight rows A1:
[[-1. -1.]
 [ 0. -1.]
 [ 1.  0.]]
Tight rows b1: [-1.  0.  1.]
Untight rows A2:
[[-1.  0.]]
Untight rows b2: [0.]
Vertex [1. 0.] is degenerate: True
Vector b after removing degeneracy: [-0.5     0.25    0.125   1.0625]
Tight rows A1:
[]
Tight rows b1: []
Untight rows A2:
[[-1. -1.]
 [-1.  0.]
 [ 0. -1.]
 [ 1.  0.]

In [17]:
import numpy as np
import pandas as pd
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        """
        Initialize the Simplex Algorithm with data from a CSV file.

        Args:
            filePath (str): Path to the CSV file containing the simplex tableau.
        """
        # Read the CSV file without headers
        arr = pd.read_csv(filePath, header=None, skip_blank_lines=True)
        
        # Extract vectors and matrix based on the input format
        self.z = arr.iloc[0, :-1].values.astype(np.float64)  # Initial feasible point (shape: (n,))
        self.c = arr.iloc[1, :-1].values.astype(np.float64)  # Cost vector (shape: (n,))
        self.b = arr.iloc[2:, -1].values.astype(np.float64)  # Constraint bounds (shape: (m,))
        self.A = arr.iloc[2:, :-1].values.astype(np.float64)  # Constraint matrix (shape: (m, n))
        
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints
        
        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")
        
        self.b_original = self.b.copy()  # Save the original b for perturbation
        self.epsilon = 1e-5  # Initial perturbation factor to handle degeneracy

        # Debugging prints
        print(f"Initialized Simplex Algorithm with input data:\n"
              f"Initial feasible point z: {self.z}\n"
              f"Cost vector c: {self.c}\n"
              f"Constraint matrix A:\n{self.A}\n"
              f"Constraint vector b: {self.b}\n")

    def isFeasible(self, z=None):
        """
        Check if the current solution z is feasible.

        Args:
            z (np.ndarray, optional): Solution vector to check. Defaults to self.z.

        Returns:
            bool: True if feasible, False otherwise.
        """
        if z is None:
            z = self.z
        feasibility = np.all(np.matmul(self.A, z) <= self.b + 1e-8)  # Added tolerance for numerical stability
        print(f"Checking feasibility of z={z} -> Feasible: {feasibility}")
        return feasibility

    def __classify_constraints(self, z):
        """
        Classify constraints as active or inactive based on the current z.

        Args:
            z (np.ndarray): Current solution vector.
        """
        constraint_values = np.matmul(self.A, z)
        tolerance = 1e-6  # Tolerance for determining active constraints
        active_indices = np.where(np.abs(constraint_values - self.b) <= tolerance)[0]
        inactive_indices = np.where(constraint_values < self.b - tolerance)[0]
        
        self.A1 = self.A[active_indices]
        self.b1 = self.b[active_indices]
        self.A2 = self.A[inactive_indices]
        self.b2 = self.b[inactive_indices]
        self.active_indices = active_indices
        self.inactive_indices = inactive_indices
        
        print(f"Classified constraints for z={z}:\n"
              f"Active Constraints A1:\n{self.A1}\n"
              f"Active Bounds b1: {self.b1}\n"
              f"Inactive Constraints A2:\n{self.A2}\n"
              f"Inactive Bounds b2: {self.b2}\n")

    def __isVertex(self, z):
        """
        Check if the current solution z is a vertex.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if z is a vertex, False otherwise.
        """
        self.__classify_constraints(z)
        rank = np.linalg.matrix_rank(self.A1)
        print(f"Checking if z={z} is a vertex -> Rank of A1: {rank}, Required: {self.n}")
        return rank == self.n

    def __testDegeneracy(self, z):
        """
        Test if the current vertex is degenerate.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if degenerate, False otherwise.
        """
        self.__classify_constraints(z)
        rank = np.linalg.matrix_rank(self.A1)
        is_degenerate = rank < self.A1.shape[0]
        print(f"Testing degeneracy for z={z} -> Rank: {rank}, "
              f"Number of active constraints: {self.A1.shape[0]} -> {'Degenerate' if is_degenerate else 'Non-degenerate'}")
        return is_degenerate

    def __removeDegeneracy(self):
        """
        Perturb b slightly to handle degeneracy by adding a small epsilon to active constraints.
        """
        perturbation = self.epsilon ** np.arange(1, len(self.active_indices) + 1)
        self.b[self.active_indices] = self.b_original[self.active_indices] + perturbation
        print(f"Perturbed b for active constraints: {self.b[self.active_indices]} with epsilon={self.epsilon}")
        self.epsilon *= 2  # Increase epsilon for future perturbations if needed

    def __testOptimum(self, z):
        """
        Test if the current vertex is optimal.

        Args:
            z (np.ndarray): Current solution vector.

        Returns:
            bool: True if optimal, False otherwise.
        """
        self.__classify_constraints(z)

        try:
            A1_inv = np.linalg.inv(self.A1)
        except np.linalg.LinAlgError:
            raise ValueError("Active constraint matrix A1 is singular.")

        beta = np.dot(A1_inv.T, self.c)
        print(f"Testing optimality for z={z} -> Beta values: {beta}")
        is_optimal = np.all(beta >= -1e-6)  # Using a small tolerance for numerical stability
        print(f"Optimality test result -> {'Optimal' if is_optimal else 'Not optimal'}")
        return is_optimal

    def solve(self):
        """
        Solve the linear programming problem using the simplex algorithm.
        """
        if not self.isFeasible():
            print(f"Initial point {self.z} is not feasible!")
            return

        solved = False
        iteration = 0
        max_iterations = 1000  # Prevent infinite loops
        z = self.z.copy()

        while not solved and iteration < max_iterations:
            iteration += 1
            print(f"\n--- Iteration {iteration} ---")

            # Move to a valid vertex if not at one already
            if not self.__isVertex(z):
                print(f"Moving to a vertex from z={z}...")
                z = self.__moveToVertex(z)

            # Test for degeneracy
            if self.__testDegeneracy(z):
                print(f"Degenerate vertex detected at z={z}. Applying perturbation...")
                self.__removeDegeneracy()
                continue

            print(f"Current Vertex z={z} -> Cost: {np.dot(self.c, z)}")

            # Test if the current vertex is optimal
            if self.__testOptimum(z):
                print(f"Optimal solution found at z={z} -> Cost: {np.dot(self.c, z)}")
                solved = True
                break

            # Try to optimize further
            z, continue_opt = self.__optimize(z)
            if not continue_opt:
                print(f"Optimal solution confirmed at z={z} -> Cost: {np.dot(self.c, z)}")
                solved = True

        if iteration == max_iterations:
            print("Maximum iterations reached. Possible cycling or no feasible solution.")
            return

        print(f"Final solution: Vertex z={z} -> Cost: {np.dot(self.c, z)}")

# Example Usage
if __name__ == "__main__":
    # Replace the file path with the path to your CSV input file
    file_path = r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv'
    simplex = SimplexAlgo(file_path)
    simplex.solve()

Initialized Simplex Algorithm with input data:
Initial feasible point z: [1. 2.]
Cost vector c: [-3. -4.]
Constraint matrix A:
[[-1. -1.]
 [-1.  0.]
 [ 0. -1.]
 [ 1.  0.]]
Constraint vector b: [-1.  0.  0.  1.]

Checking feasibility of z=[1. 2.] -> Feasible: True

--- Iteration 1 ---
Classified constraints for z=[1. 2.]:
Active Constraints A1:
[[1. 0.]]
Active Bounds b1: [1.]
Inactive Constraints A2:
[[-1. -1.]
 [-1.  0.]
 [ 0. -1.]]
Inactive Bounds b2: [-1.  0.  0.]

Checking if z=[1. 2.] is a vertex -> Rank of A1: 1, Required: 2
Moving to a vertex from z=[1. 2.]...


AttributeError: 'SimplexAlgo' object has no attribute '_SimplexAlgo__moveToVertex'

In [ ]:
import numpy as np
import pandas as pd

def simplex_algorithm(csv_file):
    """
    Simplex algorithm to maximize the objective function given constraints.
    
    Parameters:
    csv_file (str): Path to the CSV file with m+2 rows and n+1 columns.
    
    Output:
    Prints the sequence of vertices visited and the value of the objective function at each vertex.
    """
    # Load the CSV file and parse the data
    data = pd.read_csv(csv_file, header=None).values
    z = data[0, :-1]  # Initial feasible point
    c = data[1, :-1]  # Cost vector
    b = data[2:, -1]  # Constraint vector
    A = data[2:, :-1]  # Coefficient matrix
    
    # Number of variables and constraints
    m, n = A.shape
    
    # Phase I: Construct the initial tableau
    tableau = np.zeros((m + 1, n + m + 1))
    tableau[:-1, :n] = A  # Coefficient matrix
    tableau[:-1, n:n + m] = np.eye(m)  # Slack variables
    tableau[:-1, -1] = b  # Constraint vector
    tableau[-1, :n] = -c  # Cost function
    
    # Sequence of vertices visited
    vertices = []
    
    # Phase II: Perform Simplex iterations
    while True:
        # Check if optimal solution is reached (no negative cost coefficients)
        if all(tableau[-1, :-1] >= 0):
            break
        
        # Pivot column (entering variable)
        pivot_col = np.argmin(tableau[-1, :-1])
        
        # Calculate ratios for the pivot row (leaving variable)
        ratios = []
        for i in range(m):
            if tableau[i, pivot_col] > 0:
                ratios.append(tableau[i, -1] / tableau[i, pivot_col])
            else:
                ratios.append(np.inf)
        
        pivot_row = np.argmin(ratios)
        
        # If all ratios are infinite, the problem is unbounded
        if np.all(np.isinf(ratios)):
            print("The problem is unbounded.")
            return
        
        # Perform the pivot operation
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element
        for i in range(m + 1):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]
        
        # Record the current vertex
        vertex = np.zeros(n)
        for i in range(n):
            col = tableau[:-1, i]
            if np.count_nonzero(col) == 1 and np.sum(col) == 1:
                row = np.where(col == 1)[0][0]
                vertex[i] = tableau[row, -1]
        vertices.append((vertex, -tableau[-1, -1]))  # Store vertex and objective value
    
    # Print the sequence of vertices visited
    print("Sequence of vertices visited and objective values:")
    for vertex, value in vertices:
        print(f"Vertex: {vertex}, Objective Value: {value}")
    

# Save this function to a Python script and call `simplex_algorithm("path_to_csv_file.csv")`



In [19]:
simplex_algorithm(r"C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv")

ValueError: Initial solution is not feasible. Ensure the input data satisfies the constraints.

In [20]:
import numpy as np
import pandas as pd
from io import StringIO


In [21]:
def parse_csv_input(df):
    """
    Parses the input DataFrame to extract z, c, A, and b.

    Parameters:
    df (pd.DataFrame): Input DataFrame with m+2 rows and n+1 columns.

    Returns:
    z (np.ndarray): Initial feasible point.
    c (np.ndarray): Cost vector.
    A (np.ndarray): Constraint matrix.
    b (np.ndarray): Constraint vector.
    """
    data = df.values

    m_plus_2, n_plus_1 = data.shape
    m = m_plus_2 - 2  # Number of constraints
    n = n_plus_1 - 1   # Number of variables

    # Initial feasible point z (not used in simplex, but parsed as per specification)
    z = data[0, :-1].astype(float)

    # Cost vector c
    c = data[1, :-1].astype(float)

    # Constraint vector b
    b = data[2:, -1].astype(float)

    # Constraint matrix A
    A = data[2:, :-1].astype(float)

    return z, c, A, b

def print_results(vertices, objective_values, n, m):
    """
    Prints the sequence of vertices and their corresponding objective function values.

    Parameters:
    vertices (list of np.ndarray): List of vertices visited.
    objective_values (list of float): Corresponding objective function values.
    n (int): Number of original variables.
    m (int): Number of constraints.
    """
    print("Sequence of vertices visited and their objective function values:")
    for idx, (vertex, z) in enumerate(zip(vertices, objective_values)):
        var_values = vertex[:n]  # Assuming first n variables are original variables
        print(f"Step {idx + 1}: Vertex {var_values}, Objective Value = {z}")


In [22]:
def simplex(c, A, b):
    """
    Performs the simplex algorithm using the tableau method with Bland's Rule
    to handle degeneracy.

    Parameters:
    c (np.ndarray): Cost vector.
    A (np.ndarray): Constraint matrix.
    b (np.ndarray): Constraint vector.

    Returns:
    vertices (list of np.ndarray): Sequence of vertices visited.
    objective_values (list of float): Objective function values at each vertex.
    """
    m, n = A.shape

    # Initialize the tableau
    # The tableau has m + 1 rows and n + m + 1 columns
    # Last row is the objective function
    # Slack variables are added to convert inequalities to equalities
    tableau = np.zeros((m + 1, n + m + 1))

    # Fill constraint coefficients and slack variables
    tableau[:m, :n] = A
    tableau[:m, n:n+m] = np.eye(m)
    tableau[:m, -1] = b

    # Fill the objective function
    tableau[m, :n] = -c

    # Basis: Initially, slack variables are the basic variables
    basis = list(range(n, n + m))

    vertices = []
    objective_values = []

    while True:
        # Record current basic variables' values
        basic_vars = tableau[:m, -1]
        current_vertex = np.zeros(n + m)
        for i in range(m):
            current_vertex[basis[i]] = basic_vars[i]
        vertices.append(current_vertex.copy())

        # Compute current objective value
        z = tableau[m, -1]
        objective_values.append(z)

        # Check for optimality (no negative coefficients in objective row)
        # Use a small epsilon to handle numerical precision
        epsilon = 1e-8
        if all(tableau[m, :-1] >= -epsilon):
            break

        # Determine entering variable using Bland's Rule:
        # Choose the smallest index with a negative coefficient in the objective row
        entering_candidates = [j for j in range(n + m) if tableau[m, j] < -epsilon]
        if not entering_candidates:
            break  # Optimal

        entering = min(entering_candidates)

        # Determine leaving variable using minimum ratio test with Bland's Rule
        ratios = []
        for i in range(m):
            if tableau[i, entering] > epsilon:
                ratios.append(tableau[i, -1] / tableau[i, entering])
            else:
                ratios.append(np.inf)

        min_ratio = min(ratios)
        leaving_candidates = [i for i, ratio in enumerate(ratios) if ratio == min_ratio]

        if not leaving_candidates:
            raise Exception("Linear program is unbounded.")

        # Choose the leaving variable with the smallest index (Bland's Rule)
        leaving = min(leaving_candidates)

        # Pivot
        pivot = tableau[leaving, entering]
        tableau[leaving, :] = tableau[leaving, :] / pivot
        for i in range(m + 1):
            if i != leaving:
                tableau[i, :] = tableau[i, :] - tableau[i, entering] * tableau[leaving, :]

        # Update basis
        basis[leaving] = entering

    return vertices, objective_values


In [23]:
# Parse the input data
file_path = r"C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv"
df = pd.read_csv(file_path, header=None)

z, c, A, b = parse_csv_input(df)

m, n = A.shape

# Perform the simplex algorithm
vertices, objective_values = simplex(c, A, b)

# Print the results
print_results(vertices, objective_values, n, m)


Sequence of vertices visited and their objective function values:
Step 1: Vertex [0. 0.], Objective Value = 0.0


In [24]:
import numpy as np
import pandas as pd

def simplex_algorithm(file_path):
    # Read and parse the CSV file
    data = pd.read_csv(file_path, header=None)
    z = data.iloc[0, :-1].values
    c = data.iloc[1, :-1].values
    b = data.iloc[2:, -1].values
    A = data.iloc[2:, :-1].values

    # Number of variables and constraints
    num_constraints, num_variables = A.shape

    # Create initial tableau
    tableau = np.zeros((num_constraints + 1, num_variables + num_constraints + 1))
    tableau[:num_constraints, :num_variables] = A
    tableau[:num_constraints, num_variables:num_variables + num_constraints] = np.eye(num_constraints)
    tableau[:num_constraints, -1] = b
    tableau[-1, :num_variables] = -c

    # Track vertices visited and their objective values
    vertices = []
    objective_values = []

    while True:
        # Check for optimality
        if np.all(tableau[-1, :-1] >= 0):
            print("Optimal solution found.")
            break

        # Determine entering variable (pivot column)
        pivot_col = np.argmin(tableau[-1, :-1])

        # Check for unboundedness
        if np.all(tableau[:-1, pivot_col] <= 0):
            print("Problem is unbounded.")
            return vertices, objective_values

        # Determine leaving variable (pivot row)
        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios[tableau[:-1, pivot_col] <= 0] = np.inf  # Avoid division by zero or negative values
        pivot_row = np.argmin(ratios)

        # Pivot operation
        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element
        for i in range(tableau.shape[0]):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

        # Track current vertex and objective value
        current_vertex = np.zeros(num_variables)
        for i in range(num_constraints):
            col = np.where(tableau[i, :num_variables] == 1)[0]
            if len(col) == 1:
                current_vertex[col[0]] = tableau[i, -1]
        vertices.append(current_vertex)
        objective_values.append(-tableau[-1, -1])

    return vertices, objective_values


# Example usage
file_path = r"C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv"  # Replace with the actual file path
vertices, obj_values = simplex_algorithm(file_path)

# Print results
print("Visited Vertices:")
for vertex, value in zip(vertices, obj_values):
    print(f"Vertex: {vertex}, Objective Value: {value}")


Optimal solution found.
Visited Vertices:


In [2]:
def two_phase_simplex_with_bland(file_path):
    import pandas as pd
    import numpy as np

    # Read and parse the CSV file
    data = pd.read_csv(file_path, header=None)
    c = data.iloc[1, :-1].values  # Cost vector
    b = data.iloc[2:, -1].values  # RHS of constraints
    A = data.iloc[2:, :-1].values  # Constraint matrix

    num_constraints, num_variables = A.shape

    # Phase 1: Add artificial variables
    tableau = np.zeros((num_constraints + 1, num_variables + num_constraints + 1))
    tableau[:num_constraints, :num_variables] = A
    tableau[:num_constraints, num_variables:num_variables + num_constraints] = np.eye(num_constraints)
    tableau[:num_constraints, -1] = b
    tableau[-1, num_variables:num_variables + num_constraints] = -1  # Artificial variable objective

    if np.any(b < 0):
        print("Adjusting for negative b values.")
        for i in range(num_constraints):
            if b[i] < 0:
                tableau[i, :] *= -1

    # Phase 1: Solve for feasibility using Bland's Rule
    basic_vars = list(range(num_variables, num_variables + num_constraints))
    while True:
        if np.all(tableau[-1, :-1] >= 0):
            print("Phase 1 complete.")
            break

        # Bland's Rule: Choose smallest index for entering variable
        pivot_col = np.where(tableau[-1, :-1] < 0)[0]
        if len(pivot_col) == 0:
            print("Original problem is infeasible.")
            return [], []

        pivot_col = pivot_col[0]  # Bland's Rule: Choose smallest index

        # Compute ratios for leaving variable
        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios[tableau[:-1, pivot_col] <= 0] = np.inf
        if np.all(ratios == np.inf):
            print("Original problem is infeasible.")
            return [], []

        pivot_row = np.where(ratios == np.min(ratios))[0][0]  # Bland's Rule: Smallest index

        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element
        for i in range(tableau.shape[0]):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

        basic_vars[pivot_row] = pivot_col

    # Check if artificial variables have been eliminated
    if not np.isclose(tableau[-1, -1], 0):
        print("Original problem is infeasible.")
        return [], []

    # Prepare for Phase 2
    tableau = tableau[:, :num_variables + num_constraints + 1]
    tableau[-1, :num_variables] = -c
    tableau[-1, num_variables:num_variables + num_constraints] = 0

    # Phase 2: Solve original problem using Bland's Rule
    vertices = []
    objective_values = []

    while True:
        if np.all(tableau[-1, :-1] >= 0):
            print("Optimal solution found.")
            break

        # Bland's Rule: Choose smallest index for entering variable
        pivot_col = np.where(tableau[-1, :-1] < 0)[0]
        pivot_col = pivot_col[0]  # Bland's Rule

        # Compute ratios for leaving variable
        ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
        ratios[tableau[:-1, pivot_col] <= 0] = np.inf
        if np.all(ratios == np.inf):
            print("Problem is unbounded.")
            return vertices, objective_values

        pivot_row = np.where(ratios == np.min(ratios))[0][0]  # Bland's Rule

        pivot_element = tableau[pivot_row, pivot_col]
        tableau[pivot_row, :] /= pivot_element
        for i in range(tableau.shape[0]):
            if i != pivot_row:
                tableau[i, :] -= tableau[i, pivot_col] * tableau[pivot_row, :]

        basic_vars[pivot_row] = pivot_col

        # Track vertices
        current_vertex = np.zeros(num_variables)
        for i, var in enumerate(basic_vars):
            if var < num_variables:
                current_vertex[var] = tableau[i, -1]
        vertices.append(current_vertex)
        objective_values.append(-tableau[-1, -1])

    return vertices, objective_values


# Example usage
file_path = r"C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv"  # Replace with the actual file path

vertices, obj_values = two_phase_simplex_with_bland(file_path)

# Print results
print("\nFinal Results:")
print("Visited Vertices:")
for vertex, value in zip(vertices, obj_values):
    print(f"Vertex: {vertex}, Objective Value: {value}")



Adjusting for negative b values.
Original problem is infeasible.

Final Results:
Visited Vertices:


C:\Users\gupta\AppData\Local\Temp\ipykernel_29324\624025657.py:42: RuntimeWarning: divide by zero encountered in divide
  ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]
C:\Users\gupta\AppData\Local\Temp\ipykernel_29324\624025657.py:42: RuntimeWarning: invalid value encountered in divide
  ratios = tableau[:-1, -1] / tableau[:-1, pivot_col]


In [27]:
import numpy as np
from sympy import Matrix, zoo

class SimplexAlgorithm:
    def __init__(self, filename):
        self.filename = filename
        self.A, self.b, self.c, self.z = self._read_csv()
        self.b_original = self.b.copy()
        self.epsilon = pow(2, -1)

    def _read_csv(self):
        """Reads input data from the CSV file."""
        with open(self.filename) as file:
            z = list(map(float, file.readline().split(',')))[:-1]
            c = list(map(float, file.readline().split(',')))[:-1]
            A, b = [], []
            for line in file.readlines():
                l = list(map(float, line.split(',')))
                A.append(l[:-1])
                b.append(l[-1])
        return np.array(A, dtype=float), np.array(b, dtype=float), np.array(c, dtype=float), np.array(z, dtype=float)

    def test_feasibility(self):
        """Checks if the initial point is feasible."""
        return all(np.matmul(self.A, self.z) <= self.b)

    def split_rows(self):
        """Splits the rows of A and b into tight and untight rows."""
        A1, A2, b1, b2 = [], [], [], []
        Az = np.dot(self.A, self.z)
        for i in range(len(Az)):
            if Az[i] == self.b[i]:
                A1.append(self.A[i])
                b1.append(self.b[i])
            else:
                A2.append(self.A[i])
                b2.append(self.b[i])
        return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

    def test_vertex(self):
        """Checks if the point z is a vertex."""
        A1, _, _, _ = self.split_rows()
        return np.linalg.matrix_rank(self.A) <= A1.shape[0]

    def move(self):
        """Moves the feasible point toward a vertex."""
        A1, _, A2, b2 = self.split_rows()
        A1 = A1.reshape((A1.shape[0], self.A.shape[1]))
        
        # Compute the nullspace and ensure it is a float array
        nullspace = np.array(Matrix(A1).nullspace()).astype(np.float64)
        
        # Generate a random direction within the nullspace
        direction = np.sum(
            nullspace * np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1),
            axis=0
        )
        direction = direction / np.linalg.norm(direction)  # Normalize the direction

        # Compute step size alpha
        temp = np.dot(A2, direction)
        temp[temp == 0] = 1e-32  # Avoid division by zero
        alpha = (b2 - np.dot(A2, self.z)) / temp.T
        alpha = [a for a in alpha[0] if not ((a > 10**6) or (-a) > 10**6)]
        alpha = alpha[np.argmin(np.abs(alpha.copy()))]

        # Update the feasible point
        self.z = self.z + alpha * direction.T[0]


    def test_degeneracy(self):
        """Checks if the current vertex is degenerate."""
        A1, _, _, _ = self.split_rows()
        return np.linalg.matrix_rank(self.A) < A1.shape[0]

    def remove_degeneracy(self):
        """Removes degeneracy by adding a small positive number to b."""
        add = self.epsilon
        for i in range(len(self.b)):
            self.b[i] += add
            add *= self.epsilon

    def test_optimum(self):
        """Checks if the current vertex is optimal."""
        A1, _, _, _ = self.split_rows()
        beta = np.dot(np.linalg.inv(A1.T), self.c.T)
        return all(x >= 0 for x in beta)

    def optimize(self):
        """Optimizes the cost function by moving to the next optimal vertex."""
        A1, _, A2, b2 = self.split_rows()
        A1_inverse = np.linalg.inv(A1).T
        direction = None
        for x in A1_inverse:
            if np.dot(-x, self.c.T) > 0:
                direction = x.T
                break
        if direction is None:
            return self.z, False
        temp = np.dot(A2, direction)
        temp[temp == 0] = 1e-16
        alpha = (np.dot(A2, self.z) - b2) / temp
        alpha = [a for a in alpha if a != zoo]
        if all(a < 0 for a in alpha):
            return self.z, False
        alpha = min(a for a in alpha if a >= 0)
        self.z = self.z - alpha * direction
        return self.z, True

    def solve(self):
        """Solves the linear optimization problem."""
        if not self.test_feasibility():
            print(f"Initial point {self.z} is not feasible!")
            return
        solved = False
        while not solved:
            while not self.test_vertex():
                self.move()
            if self.test_degeneracy():
                self.remove_degeneracy()
                self.epsilon *= 2
                continue
            print(f"Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}")
            solution = True
            while solution and not self.test_optimum():
                self.z, solution = self.optimize()
                if not solution:
                    print("No solution found.")
                    return
                elif self.test_degeneracy():
                    self.remove_degeneracy()
                    self.epsilon *= 2
                    break
                print(f"Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}")
            solved = True
        print(f"Optimal Solution: Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}")

# Example Usage
if __name__ == "__main__":
    simplex = SimplexAlgorithm(r"C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\1.Dataset.csv")
    simplex.solve()


Vertex [0. 0.] --> Cost: 0.0
Vertex [2. 0.] --> Cost: 6.0
Vertex [4.14285714 1.71428571] --> Cost: 15.857142857142856


LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [30]:
import numpy as np
from sympy import Matrix
import csv

class SimplexSolver:
    def __init__(self, filename):
        """
        Initializes the SimplexSolver with data from a CSV file.

        Parameters:
        - filename (str): Path to the input CSV file.
        """
        self.filename = filename
        self.A = None
        self.b = None
        self.c = None
        self.z = None
        self.b_original = None
        self.epsilon = 2 ** -1

    def read_csv(self):
        """
        Reads the CSV file and initializes A, b, c, and z.
        """
        with open(self.filename, 'r') as input_file:
            reader = csv.reader(input_file)
            rows = list(reader)

            # Extract z from the first row (excluding the last element)
            self.z = np.array(list(map(float, rows[0][:-1])), dtype=float)

            # Extract c from the second row (excluding the last element)
            self.c = np.array(list(map(float, rows[1][:-1])), dtype=float)

            # Extract A and b from the remaining rows
            A = []
            b = []
            for row in rows[2:]:
                A.append(list(map(float, row[:-1])))
                b.append(float(row[-1]))
            self.A = np.array(A, dtype=float)
            self.b = np.array(b, dtype=float)
            self.b_original = self.b.copy()

    def test_feasibility(self, z):
        """
        Checks whether a point z is a feasible solution.

        Parameters:
        - z (np.ndarray): The point to check.

        Returns:
        - bool: True if feasible, False otherwise.
        """
        return np.all(np.dot(self.A, z) <= self.b + 1e-8)  # Adding epsilon for numerical stability

    def split_rows(self, z):
        """
        Splits A and b into tight and untight rows based on z.

        Parameters:
        - z (np.ndarray): The current point.

        Returns:
        - A1 (np.ndarray): Tight rows of A.
        - b1 (np.ndarray): Tight elements of b.
        - A2 (np.ndarray): Untight rows of A.
        - b2 (np.ndarray): Untight elements of b.
        """
        Az = np.dot(self.A, z)
        tight_indices = np.isclose(Az, self.b)
        untight_indices = ~tight_indices

        A1 = self.A[tight_indices]
        b1 = self.b[tight_indices]
        A2 = self.A[untight_indices]
        b2 = self.b[untight_indices]

        return A1, b1, A2, b2

    def test_vertex(self, z):
        """
        Checks whether z is a vertex of the feasible region.

        Parameters:
        - z (np.ndarray): The point to check.

        Returns:
        - bool: True if z is a vertex, False otherwise.
        """
        A1, _, _, _ = self.split_rows(z)
        rank_A = np.linalg.matrix_rank(self.A)
        rank_A1 = np.linalg.matrix_rank(A1)
        return rank_A1 >= rank_A

    def move(self, z):
        """
        Moves the feasible point towards a vertex by one step.

        Parameters:
        - z (np.ndarray): The current feasible point.

        Returns:
        - np.ndarray: The updated feasible point.
        """
        A1, _, A2, b2 = self.split_rows(z)
        n = self.A.shape[1]

        if A1.size == 0:
            raise ValueError("No tight constraints to determine direction.")

        nullspace = np.array(Matrix(A1).nullspace(), dtype=float).flatten()
        if nullspace.size == 0:
            raise ValueError("No nullspace found. The current point is likely a vertex.")

        # Generate a random direction in the nullspace
        scale = np.random.randint(1, 4, size=nullspace.shape)
        u = nullspace * scale
        u = u / np.linalg.norm(u)

        # Calculate step size alpha
        Au = np.dot(A2, u)
        with np.errstate(divide='ignore', invalid='ignore'):
            alpha = (self.b[~np.isclose(np.dot(self.A, z), self.b)] - np.dot(A2, z)) / Au
        alpha = alpha[Au > 1e-8]  # Consider only positive steps

        if alpha.size == 0:
            raise ValueError("No feasible direction to move.")

        alpha_min = np.min(alpha)
        z_new = z + alpha_min * u

        return z_new

    def test_degeneracy(self, z):
        """
        Checks whether the vertex z is degenerate.

        Parameters:
        - z (np.ndarray): The vertex to check.

        Returns:
        - bool: True if degenerate, False otherwise.
        """
        A1, _, _, _ = self.split_rows(z)
        rank_A = np.linalg.matrix_rank(self.A)
        rank_A1 = np.linalg.matrix_rank(A1)
        return rank_A1 > rank_A

    def remove_degeneracy(self):
        """
        Removes degeneracy by perturbing the b vector.
        """
        add = self.epsilon
        for i in range(len(self.b)):
            self.b[i] += add
            add *= self.epsilon
        self.epsilon *= 2

    def test_optimum(self, z):
        """
        Checks whether z is an optimal vertex.

        Parameters:
        - z (np.ndarray): The vertex to check.

        Returns:
        - bool: True if optimal, False otherwise.
        """
        A1, _, _, _ = self.split_rows(z)
        try:
            inv_A1 = np.linalg.inv(A1)
        except np.linalg.LinAlgError:
            return False  # Singular matrix, cannot determine optimum

        beta = np.dot(inv_A1.T, self.c)
        return np.all(beta >= -1e-8)  # Allowing a small negative tolerance

    def optimize(self, z):
        """
        Moves to the next vertex to optimize the cost function.

        Parameters:
        - z (np.ndarray): The current vertex.

        Returns:
        - tuple: (new vertex, True) if move is successful, (z, False) otherwise.
        """
        A1, _, A2, b2 = self.split_rows(z)
        try:
            inv_A1 = np.linalg.inv(A1)
        except np.linalg.LinAlgError:
            return z, False  # Cannot invert, likely degenerate

        directions = -np.dot(inv_A1.T, self.c)
        positive_directions = directions > 1e-8

        if not np.any(positive_directions):
            return z, False  # Optimal solution found

        # Select the first positive direction
        u = directions.copy()
        u[~positive_directions] = 0
        u = u / np.linalg.norm(u)

        # Calculate step size alpha
        Au = np.dot(self.A, u)
        with np.errstate(divide='ignore', invalid='ignore'):
            alpha = (self.b - np.dot(self.A, z)) / Au
        alpha = alpha[Au > 1e-8]  # Consider only positive steps

        if alpha.size == 0:
            return z, False  # No feasible direction to move

        alpha_min = np.min(alpha)
        z_new = z + alpha_min * u

        return z_new, True

    def find_optimal_solution(self):
        """
        Executes the simplex algorithm to find the optimal solution.

        Returns:
        - tuple: (optimal vertex, optimal cost)
        """
        if not self.test_feasibility(self.z):
            raise ValueError(f"Initial point {self.z} is not feasible.")

        solved = False

        while not solved:
            # Move towards a vertex
            while not self.test_vertex(self.z):
                try:
                    self.z = self.move(self.z)
                except ValueError as e:
                    print(f"Error during moving: {e}")
                    return None, None

            # Check for degeneracy
            if self.test_degeneracy(self.z):
                self.remove_degeneracy()
                print(f"Removed degeneracy. Updated b: {self.b}")
                continue

            # Display current vertex and its cost
            current_cost = np.dot(self.z, self.c)
            print(f"Vertex: {self.z} --> Cost: {current_cost}")

            # Optimize
            while not self.test_optimum(self.z):
                z_new, can_move = self.optimize(self.z)
                if not can_move:
                    print("No further movement possible. Optimization halted.")
                    return self.z, current_cost
                self.z = z_new
                current_cost = np.dot(self.z, self.c)
                print(f"Vertex: {self.z} --> Cost: {current_cost}")

            solved = True

        return self.z, np.dot(self.z, self.c)

    def get_original_solution(self, z):
        """
        Converts the solution back to the original variable space if necessary.

        Parameters:
        - z (np.ndarray): The solution vertex.

        Returns:
        - np.ndarray: The original solution.
        """
        tight_indices = np.isclose(np.dot(self.A, z), self.b_original)
        X = self.A[tight_indices]
        y = self.b_original[tight_indices]

        try:
            original_z = np.linalg.inv(X).dot(y)
            return original_z
        except np.linalg.LinAlgError:
            print("Cannot find original solution due to singular matrix.")
            return None

    def run(self):
        """
        Runs the simplex algorithm and displays the optimal solution.
        """
        self.read_csv()
        print("Matrix A:\n", self.A)
        print("Vector b:\n", self.b)
        print("Cost vector c:\n", self.c)
        print("Initial feasible point z:\n", self.z)

        optimal_z, optimal_cost = self.find_optimal_solution()

        if optimal_z is not None:
            print(f"\nOptimal Vertex: {optimal_z}")
            print(f"Optimal Cost: {optimal_cost}")

            original_solution = self.get_original_solution(optimal_z)
            if original_solution is not None:
                print(f"Original Solution: {original_solution} --> Cost: {np.dot(original_solution, self.c)}")
        else:
            print("No optimal solution found.")

# Example usage:
if __name__ == "__main__":
    # Replace 'path_to_csv.csv' with your actual CSV file path
    solver = SimplexSolver(r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\3.Dataset.csv')
    solver.run()


Matrix A:
 [[-1. -1.]
 [-1.  0.]
 [ 0. -1.]
 [ 1.  0.]]
Vector b:
 [-1.  0.  0.  1.]
Cost vector c:
 [-3. -4.]
Initial feasible point z:
 [1. 2.]
Error during moving: No feasible direction to move.
No optimal solution found.


In [33]:
import numpy as np
from sympy import Matrix, zoo

class SimplexSolver:
    def __init__(self, filename):
        self.A, self.b, self.c, self.z = self.readCSV(filename)
        self.b_org = self.b.copy()
        self.epsilon = pow(2, -1)

    def readCSV(self, filename):
        '''
        Reads the input matrix A, vectors b and c and initial point z from a CSV file.
        '''
        with open(filename) as input_file:
            # Read the first line to extract vector z
            line = input_file.readline()
            z = list(map(float, line.strip().split(',')))
            self.z = np.asarray(z[:-1], dtype=float)

            # Read the second line to extract vector c
            line = input_file.readline()
            c = list(map(float, line.strip().split(',')))
            self.c = np.asarray(c[:-1], dtype=float)

            # Read all the remaining lines to extract matrix A and vector b
            A, b = [], []
            for line in input_file.readlines():
                l = list(map(float, line.strip().split(',')))
                A.append(l[:-1])
                b.append(l[-1])
            self.A = np.asarray(A, dtype=float)
            self.b = np.asarray(b, dtype=float)

        return self.A, self.b, self.c, self.z

    def testFeasibility(self, z):
        '''
        Checks whether a point z is a feasible solution for the system or not.
        '''
        return all(np.matmul(self.A, z) <= self.b)

    def splitRows(self, z):
        '''
        Splits the matrices A and b into sets of tight and untight rows with respect to a given point z.
        '''
        A1, A2, b1, b2 = [], [], [], []
        Az = np.dot(self.A, z)
        for i in range(len(Az)):
            if np.dot(self.A[i], z) == self.b[i]:
                A1.append(self.A[i])
                b1.append(self.b[i])
            else:
                A2.append(self.A[i])
                b2.append(self.b[i])
        return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

    def testVertex(self, z):
        '''
        Checks whether a point z is a vertex of the feasible region or not.
        '''
        A1, _, _, _ = self.splitRows(z)
        return np.linalg.matrix_rank(self.A) <= A1.shape[0]

    def move(self, z):
        '''
        Moves the feasible point towards a vertex of the feasible region by one step size.
        '''
        _, n = self.A.shape[0], self.A.shape[1]
        A1, _, A2, b2 = self.splitRows(z)
        A1 = A1.reshape((A1.shape[0], n))

        # Compute the direction to move the feasible point
        nullspace = np.array(Matrix(A1).nullspace())
        scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
        vector = np.array(np.sum(nullspace * scale, axis=0)).astype(float)
        u = vector / np.linalg.norm(vector)

        # Compute the step size 'alpha'
        temp = np.dot(A2, u)
        temp[temp == 0] = 1e-32  # Avoid division by zero
        alpha = (b2 - np.dot(A2, z)) / temp.T
        alpha = alpha[0]
        alpha = np.array([a for a in alpha if not ((a > 1e6) or (-1 * a) > 1e6)])
        alpha = alpha[np.argmin(np.abs(alpha.copy()))]

        # Update the feasible point by one step size
        u = u.T
        z = z + alpha * u
        z = z[0]

        return z

    def testDegeneracy(self, z):
        '''
        Checks whether a vertex z of the feasible region is degenerate.
        '''
        A1, _, _, _ = self.splitRows(z)
        return np.linalg.matrix_rank(self.A) < A1.shape[0]

    def removeDegeneracy(self, b):
        '''
        Removes degeneracy from the feasible region by adding powers of a small positive number to vector b.
        '''
        add = self.epsilon
        for i in range(len(b)):
            b[i] += add
            add *= self.epsilon
        return b

    def testOptimum(self, z):
        '''
        Checks whether a vertex z is an optimum vertex or not.
        '''
        A1, _, _, _ = self.splitRows(z)
        beta = np.dot(np.linalg.inv(A1.T), self.c.T)
        return all([x >= 0 for x in beta])

    def optimize(self, z):
        '''
        Optimizes the cost function by moving to the next optimum vertex.
        '''
        A1, _, A2, b2 = self.splitRows(z)

        # Compute the direction to move in order to reach a more optimum vertex
        u = None
        A1_inverse = np.linalg.inv(A1)
        A1_inverse = np.transpose(A1_inverse)
        for x in A1_inverse:
            if np.dot((-1) * x, self.c.T) > 0:
                u = x.T
                break

        # Compute the step size 'alpha'
        temp = np.dot(A2, u)
        temp[temp == 0] = 1e-16  # Avoid division by zero
        alpha = (np.dot(A2, z) - b2) / temp
        alpha = [x for x in alpha if x != zoo]
        if all([a < 0 for a in alpha]):
            return z, False
        alpha = min([x for x in alpha if x >= 0])

        # Update the vertex
        z = z - alpha * u

        return z, True

    def run(self):
        '''
        Driver function which calls the subroutines according to the Simplex Algorithm.
        '''
        # Step 1: Check for feasibility of the Initial Point
        if not self.testFeasibility(self.z):
            print('Initial point {} is not feasible...!!!'.format(self.z))
            return

        solved = False

        while not solved:
            # Step 2: Move towards the first vertex
            while not self.testVertex(self.z):
                self.z = self.move(self.z)

            # Test for degeneracy of the vertex
            if self.testDegeneracy(self.z):
                self.b = self.removeDegeneracy(self.b_org.copy())
                self.epsilon *= 2
                continue

            # Print the value of the Cost function at this first vertex
            print('Vertex {} --> Cost: {}'.format(self.z, np.dot(self.z, self.c)))

            # Step 3: March through every vertex and check for optimum along with degeneracy
            solution = True
            degenerate = False
            while solution and (not self.testOptimum(self.z)):
                self.z, solution = self.optimize(self.z)
                if not solution:
                    print('No Solution')
                elif self.testDegeneracy(self.z):
                    self.b = self.removeDegeneracy(self.b_org.copy())
                    degenerate = True
                    self.epsilon *= 2
                    break
                else:
                    print('Vertex {} --> Cost: {}'.format(self.z, np.dot(self.z, self.c)))

            if not degenerate:
                solved = True

        # Convert the solution back to the original solution, if it exists
        if solution:
            X, y = [], []
            for i in range(len(self.A)):
                if np.dot(self.A[i], self.z) == self.b[i]:
                    X.append(self.A[i])
                    y.append(self.b_org[i])
            z = np.dot(np.linalg.inv(X), y)
            print('Original Solution: Vertex {} --> Cost: {}'.format(z, np.dot(z, self.c)))

# Example usage:
solver = SimplexSolver(r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\Some Ones assig\1.Dataset.csv')
solver.run()


Vertex [0. 0.] --> Cost: 0.0
Vertex [2. 0.] --> Cost: 6.0
Vertex [4.14285714 1.71428571] --> Cost: 15.857142857142856


LinAlgError: 1-dimensional array given. Array must be at least two-dimensional

In [40]:
import numpy as np
from sympy import Matrix, zoo

class SimplexSolver:
    def __init__(self, filename):
        self.A, self.b, self.c, self.z = self._read_csv(filename)
        self.b_org = self.b.copy()
        self.epsilon = 0.5  # Equivalent to pow(2, -1)

    def _read_csv(self, filename):
        '''
        Reads the input matrix A, vectors b and c, and initial point z from a CSV file.
        '''
        try:
            with open(filename, 'r') as input_file:
                # Read the first line to extract vector z
                line = input_file.readline()
                if not line:
                    raise ValueError('Input file is empty or does not contain initial point z.')
                z = list(map(float, line.strip().split(',')))
                z = np.asarray(z[:-1], dtype=float)

                # Read the second line to extract vector c
                line = input_file.readline()
                if not line:
                    raise ValueError('Input file does not contain cost vector c.')
                c = list(map(float, line.strip().split(',')))
                c = np.asarray(c[:-1], dtype=float)

                # Read all the remaining lines to extract matrix A and vector b
                A, b = [], []
                for line in input_file:
                    l = list(map(float, line.strip().split(',')))
                    A.append(l[:-1])
                    b.append(l[-1])
                if not A or not b:
                    raise ValueError('Input file does not contain matrix A and vector b.')

                A = np.asarray(A, dtype=float)
                b = np.asarray(b, dtype=float)

            return A, b, c, z
        except FileNotFoundError:
            raise FileNotFoundError(f'File "{filename}" not found.')
        except Exception as e:
            raise ValueError(f'Error reading file "{filename}": {e}')

    def _test_feasibility(self, z):
        '''
        Checks whether a point z is a feasible solution for the system or not.
        '''
        return np.all(np.matmul(self.A, z) <= self.b)

    def _split_rows(self, z):
        '''
        Splits the matrices A and b into sets of tight and untight rows with respect to a given point z.
        '''
        A1, A2, b1, b2 = [], [], [], []
        Az = np.dot(self.A, z)
        for i in range(len(Az)):
            if np.isclose(np.dot(self.A[i], z), self.b[i]):
                A1.append(self.A[i])
                b1.append(self.b[i])
            else:
                A2.append(self.A[i])
                b2.append(self.b[i])
        return np.array(A1), np.array(b1), np.array(A2), np.array(b2)

    def _test_vertex(self, z):
        '''
        Checks whether a point z is a vertex of the feasible region or not.
        '''
        A1, _, _, _ = self._split_rows(z)
        rank_A = np.linalg.matrix_rank(self.A)
        num_tight_rows = A1.shape[0]
        return rank_A <= num_tight_rows

    def _move(self, z):
        '''
        Moves the feasible point towards a vertex of the feasible region by one step size.
        '''
        try:
            m, n = self.A.shape
            A1, _, A2, b2 = self._split_rows(z)
            A1 = A1.reshape((A1.shape[0], n))

            # Compute the direction to move the feasible point
            nullspace = np.array(Matrix(A1).nullspace())
            if nullspace.size == 0:
                raise ValueError('Nullspace of A1 is empty; cannot move towards vertex.')
            scale = np.random.randint(1, 4, nullspace.shape[0]).reshape(nullspace.shape[0], 1, 1)
            vector = np.array(np.sum(nullspace * scale, axis=0)).astype(float)
            u = vector / np.linalg.norm(vector)

            # Compute the step size 'alpha'
            temp = np.dot(A2, u)
            temp[np.isclose(temp, 0)] = 1e-32  # Avoid division by zero
            alpha_values = (b2 - np.dot(A2, z)) / temp.T
            alpha_values = alpha_values[0]
            alpha_values = np.array([a for a in alpha_values if not ((a > 1e6) or (-a) > 1e6)])
            if alpha_values.size == 0:
                raise ValueError('No valid alpha found; cannot move towards vertex.')
            alpha = alpha_values[np.argmin(np.abs(alpha_values.copy()))]

            # Update the feasible point by one step size
            u = u.T
            z_new = z + alpha * u
            z_new = z_new[0]

            return z_new
        except Exception as e:
            raise RuntimeError(f'Error in moving towards vertex: {e}')

    def _test_degeneracy(self, z):
        '''
        Checks whether a vertex z of the feasible region is degenerate.
        '''
        A1, _, _, _ = self._split_rows(z)
        return np.linalg.matrix_rank(self.A) < A1.shape[0]

    def _remove_degeneracy(self, b):
        '''
        Removes degeneracy from the feasible region by adding powers of a small positive number to vector b.
        '''
        add = self.epsilon
        for i in range(len(b)):
            b[i] += add
            add *= self.epsilon
        return b

    def _test_optimum(self, z):
        '''
        Checks whether a vertex z is an optimum vertex or not.
        '''
        try:
            A1, _, _, _ = self._split_rows(z)
            beta = np.dot(np.linalg.inv(A1.T), self.c.T)
            return np.all(beta >= 0)
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A1.T is singular; cannot compute beta.')
        except Exception as e:
            raise RuntimeError(f'Error in testing for optimum: {e}')

    def _optimize(self, z):
        '''
        Optimizes the cost function by moving to the next optimum vertex.
        '''
        try:
            A1, _, A2, b2 = self._split_rows(z)

            # Compute the direction to move in order to reach a more optimum vertex
            A1_inverse = np.linalg.inv(A1)
            A1_inverse = np.transpose(A1_inverse)
            u = None
            for x in A1_inverse:
                if np.dot(-x, self.c.T) > 0:
                    u = x.T
                    break
            if u is None:
                raise ValueError('No direction found to optimize the cost function.')

            # Compute the step size 'alpha'
            temp = np.dot(A2, u)
            temp[np.isclose(temp, 0)] = 1e-16  # Avoid division by zero
            alpha_values = (np.dot(A2, z) - b2) / temp
            alpha_values = [x for x in alpha_values if x != zoo]
            if all([a < 0 for a in alpha_values]):
                return z, False
            alpha = min([x for x in alpha_values if x >= 0])

            # Update the vertex
            z_new = z - alpha * u

            return z_new, True
        except np.linalg.LinAlgError:
            raise np.linalg.LinAlgError('Matrix A1 is singular; cannot compute inverse.')
        except Exception as e:
            raise RuntimeError(f'Error in optimization step: {e}')

    def solve(self):
        '''
        Driver function which calls the subroutines according to the Simplex Algorithm.
        '''
        try:
            # Step 1: Check for feasibility of the Initial Point
            if not self._test_feasibility(self.z):
                raise ValueError(f'Initial point {self.z} is not feasible.')

            solved = False

            while not solved:
                # Step 2: Move towards the first vertex
                while not self._test_vertex(self.z):
                    self.z = self._move(self.z)

                # Test for degeneracy of the vertex
                if self._test_degeneracy(self.z):
                    self.b = self._remove_degeneracy(self.b_org.copy())
                    self.epsilon *= 2
                    continue

                # Print the value of the Cost function at this first vertex
                print(f'Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}')

                # Step 3: March through every vertex and check for optimum along with degeneracy
                solution = True
                degenerate = False
                while solution and (not self._test_optimum(self.z)):
                    self.z, solution = self._optimize(self.z)
                    if not solution:
                        raise ValueError('No solution found during optimization.')
                    elif self._test_degeneracy(self.z):
                        self.b = self._remove_degeneracy(self.b_org.copy())
                        degenerate = True
                        self.epsilon *= 2
                        break
                    else:
                        print(f'Vertex {self.z} --> Cost: {np.dot(self.z, self.c)}')

                if not degenerate:
                    solved = True

            # Convert the solution back to the original solution, if it exists
            if solution:
                X, y = [], []
                for i in range(len(self.A)):
                    if np.isclose(np.dot(self.A[i], self.z), self.b[i]):
                        X.append(self.A[i])
                        y.append(self.b_org[i])
                X = np.array(X)
                y = np.array(y)
                try:
                    z_solution = np.linalg.solve(X, y)
                except np.linalg.LinAlgError:
                    raise np.linalg.LinAlgError('Final system is singular; cannot find unique solution.')
                print(f'Original Solution: Vertex {z_solution} --> Cost: {np.dot(z_solution, self.c)}')
        except Exception as e:
            print(f'Error: {e}')


In [41]:
if __name__ == '__main__':
    # Replace the filename with the path to your input CSV file
    filename = r'neha.csv'
    solver = SimplexSolver(filename)
    solver.solve()


Vertex [-0.1875  4.25  ] --> Cost: -16.4375
Vertex [0.375 3.125] --> Cost: -13.625
Vertex [ 3.625 -0.125] --> Cost: -10.375
Original Solution: Vertex [ 4. -0.] --> Cost: -12.0
